In [7]:
import tensorflow as tf
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [2]:
import numpy as np
import sklearn as sk

In [3]:
X_train=X_train.astype(np.float32).reshape(-1,28*28)/255
X_test= X_test.astype(np.float32).reshape(-1, 28*28) / 255

# Softmax Regression

$$s_k(x)={\theta_k}^T\cdot x$$

## use sklearn

```py
from sklearn.linear_model import LogisticRegression
softmax=LogisticRegression(multi_class='multinomial',solver='lbfgs',C=10)
softmax.fit(X_train,y_train)
```

In [13]:
softmax=joblib.load('./saved_models/softmax_on_mnist.pkl')

In [14]:
from sklearn.metrics import accuracy_score
y_pred_sm=softmax.predict(X_test)
print(accuracy_score(y_test,y_pred_sm))
#0.9258

0.9258


```python
#cross validation
from sklearn.model_selection import cross_val_predict
y_scores_sm=cross_val_predict(softmax,X_train_sm,y_train,cv=3,method='predict')
method : string, optional, default: ‘predict’

from sklearn.metrics import accuracy_score
print(accuracy_score(y_train,y_scores_sm))
#0.91685
```

In [12]:
from sklearn.externals import joblib
joblib.dump(softmax,'./saved_models/softmax_on_mnist.pkl')

['./saved_models/softmax_on_mnist.pkl']

## use tensorflow

$$s_k(x)={\theta_x}^T\cdot x+b$$

In [1]:
import tensorflow as tf
import numpy as np

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist =input_data.read_data_sets('./my_datasets_mnist/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./my_datasets_mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./my_datasets_mnist/train-labels-idx1-ubyte.gz
Extracting ./my_datasets_mnist/t10k-images-idx3-ubyte.gz
Extracting ./my_datasets_mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
n_inputs=28*28

In [4]:
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
y=tf.placeholder(tf.int64,shape=(None),name='y')

In [5]:
stddev=np.sqrt(2/(n_inputs+10))
W_init=tf.truncated_normal([n_inputs,10],stddev=stddev)
W=tf.Variable(W_init)
b=tf.Variable(np.ones(10).astype(np.float32))

In [6]:
logits=tf.matmul(X,W)+b

In [7]:
entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
loss=tf.reduce_mean(entropy)

```py
#maybe another way
y_pro=tf.nn.softmax(logits)
#y = tf.placeholder(tf.float, [None,10])
entropy = -tf.reduce_sum(y*tf.log(y_pro))
loss=entropy
```


Q: Why not use $-accuracy$ as loss function?  
A: Accuracy is based on test samples.

In [65]:
accuracy=tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits,y,1),tf.float32))

In [8]:
#another way
accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits,axis=1),y),tf.float32))

In [9]:
training_op=tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [10]:
init=tf.global_variables_initializer()
n_epochs=50
batch_size=50

In [11]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            training_op.run(feed_dict={X:X_batch,
                                       y:y_batch})
            #print(y_batch)
            #print(logits.eval(feed_dict={X:X_batch,y:y_batch}))
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print(epoch,"Test accuracy:",acc_test)
    

0 Test accuracy: 0.8712
1 Test accuracy: 0.8858
2 Test accuracy: 0.8945
3 Test accuracy: 0.9003
4 Test accuracy: 0.9024
5 Test accuracy: 0.9056
6 Test accuracy: 0.9071
7 Test accuracy: 0.9097
8 Test accuracy: 0.9104
9 Test accuracy: 0.9111
10 Test accuracy: 0.9123
11 Test accuracy: 0.9133
12 Test accuracy: 0.9134
13 Test accuracy: 0.9138
14 Test accuracy: 0.9144
15 Test accuracy: 0.9146
16 Test accuracy: 0.9158
17 Test accuracy: 0.9155
18 Test accuracy: 0.9149
19 Test accuracy: 0.9167
20 Test accuracy: 0.9168
21 Test accuracy: 0.9172
22 Test accuracy: 0.9172
23 Test accuracy: 0.9182
24 Test accuracy: 0.9195
25 Test accuracy: 0.9179
26 Test accuracy: 0.9181
27 Test accuracy: 0.9184
28 Test accuracy: 0.9184
29 Test accuracy: 0.9192
30 Test accuracy: 0.9201
31 Test accuracy: 0.921
32 Test accuracy: 0.9204
33 Test accuracy: 0.9199
34 Test accuracy: 0.9204
35 Test accuracy: 0.9199
36 Test accuracy: 0.9197
37 Test accuracy: 0.9204
38 Test accuracy: 0.9213
39 Test accuracy: 0.9209
40 Test acc

In [68]:
X_batch.shape

(50, 784)

# Ensemble Learning with ANNs

# DNN